GLiNER is a Named Entity Recognition (NER) model capable of identifying any entity type using a bidirectional transformer encoder (BERT-like). It provides a practical alternative to traditional NER models, which are limited to predefined entities, and Large Language Models (LLMs) that, despite their flexibility, are costly and large for resource-constrained scenarios.

https://github.com/urchade/GLiNER <br>
https://huggingface.co/urchade/gliner_medium-v2.1

In [9]:
import pandas as pd
from gliner import GLiNER

# Załaduj model z Hugging Face
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

texts = [
"""
1. Użyte w Regulaminie określenia oznaczają:
1) dopuszczalne saldo debetowe – ujemne saldo rachunku oszczędnościowo-rozliczeniowego w wysokości nieprzekraczającej miesięcznego limitu ustalanego
przez PKO Bank Polski SA, z którego Posiadacz rachunku może korzystać na podstawie umowy rachunku,
2) data waluty – dzień, od którego lub do którego, naliczane są odsetki od środków pieniężnych, którymi został obciążony lub uznany rachunek,
3) dyspozycja – wszelkie oświadczenia woli lub wiedzy, w tym dotyczące operacji, składane przez Klienta,
4) dzień roboczy – dzień przypadający od poniedziałku do piątku, który nie jest dniem ustawowo wolnym od pracy,
5) kapitalizacja – dopisanie do salda rachunku naliczonych za dany okres odsetek od środków pieniężnych zgromadzonych na tym rachunku
""",
"""
1. PKO Bank Polski SA może prowadzić na rzecz Klientów rachunki bankowe, o których mowa w § 1. Rachunki te nie są przeznaczone do przeprowadzania rozliczeń
pieniężnych związanych z prowadzoną przez Klienta działalnością gospodarczą lub zarobkową.
2. PKO Bank Polski SA może otwierać kolejne rachunki na podstawie umowy o prowadzenie rachunków bankowych.
3. Zawarcie umowy rachunku może nastąpić:
1) za pośrednictwem elektronicznych kanałów dostępu, o ile PKO Bank Polski SA dla danego kanału udostępnia odpowiednią funkcjonalność,
2) w placówce,
3) w inny uzgodniony sposób.
4. Rachunek może być otwarty:
1) jako rachunek indywidualny - dla rezydenta albo nierezydenta,
2) jako rachunek wspólny – dla dwóch osób o tym samym statusie dewizowym, posiadających pełną zdolność do czynności prawnych, które posiadają ten
sam kraj rezydencji podatkowej i obie legitymują się ważnymi certyfikatami rezydencji podatkowej, bądź obie nie posiadają ważnych certyfikatów rezydencji
podatkowej
""",
"""
1. Posiadacz rachunku, który nie ukończył 13 lat, albo osoba ubezwłasnowolniona całkowicie nie może samodzielnie dysponować środkami pieniężnymi
zgromadzonymi na rachunku, ani dokonywać samodzielnie żadnych czynności związanych z posiadaniem rachunku.
2. W imieniu Posiadacza rachunku, o którym mowa w ust. 1, do dysponowania środkami pieniężnymi zgromadzonymi na rachunku w granicach zwykłego zarządu,
uprawniony jest jego przedstawiciel ustawowy, który zawarł umowę rachunku w imieniu Posiadacza rachunku. Dysponowanie środkami pieniężnymi w zakresie
przekraczającym zwykły zarząd, wymaga zezwolenia sądu opiekuńczego
""",
"""
1. Za usługi świadczone przez PKO Bank Polski SA, w tym za czynności związane z prowadzeniem rachunku, wysyłaniem powiadomień, a także za inne czynności,
PKO Bank Polski SA pobiera prowizje i opłaty bankowe, zgodnie z Taryfą.
2. PKO Bank Polski SA należne prowizje i opłaty bankowe, o których mowa w ust. 1, pobierać będzie, z uwzględnieniem ust. 3-4, w ciężar rachunku, za wyjątkiem
rachunku lokaty.
3. W przypadku rachunku lokaty należne prowizje i opłaty bankowe PKO Bank Polski SA pobierać będzie z rachunku, o którym mowa w § 25 ust. 2 albo w formie
gotówkowej.
4. W przypadku braku wolnych środków na rachunku, pobranie:
1) prowizji i opłat, lub
2) zaliczek na podatek dochodowy oraz składek na ubezpieczenie zdrowotne od realizowanych przez PKO Bank Polski SA rent i emerytur z zagranicy
może spowodować powstanie zadłużenia przeterminowanego.
""",
"""
1. Środki pieniężne zgromadzone na rachunku mogą być oprocentowane zgodnie z zasadami określonymi w Regulaminie lub w umowie rachunku.
2. W przypadku, gdy środki pieniężne zgromadzone na rachunku są oprocentowane według zmiennej stopy procentowej, w okresie obowiązywania umowy
rachunku, PKO Bank Polski SA jest uprawniony do zmiany wysokości oprocentowania środków pieniężnych zgromadzonych na tym rachunku na zasadach
określonych w ust. 3.
"""
]

# Rozszerzona lista etykiet dla sektora bankowego
labels = [
    "kredyt hipoteczny", "pożyczka gotówkowa", "konto osobiste", "konto firmowe", "konto oszczędnościowe", "lokata terminowa",
    "ubezpieczenie", "kredyt firmowy", "kredyt konsolidacyjny", "karta kredytowa", "karta debetowa", "bankomat",
    "oddział bankowy", "opłaty", "prowizja", "transakcje", "limit kredytowy", "umowa kredytowa", "oprocentowanie",
    "kapitał", "odsetki", "raty", "czyszczenie historii kredytowej", "ubezpieczenie nieruchomości", "ubezpieczenie na życie",
    "rachunek wspólny", "rachunek maklerski", "inwestycje", "fundusze inwestycyjne", "waluty", "przelewy międzynarodowe",
    "spłata kredytu", "oferta promocyjna", "depozyt", "czek", "weksel", "gwarancje bankowe", "certyfikat depozytowy",
    "emisja obligacji", "portfel inwestycyjny", "analiza kredytowa", "scoring kredytowy", "zdolność kredytowa",
    "windykacja", "restrukturyzacja kredytu", "odroczenie płatności", "plan oszczędnościowy", "konto młodzieżowe",
    "konto walutowe", "platforma bankowości elektronicznej", "aplikacja mobilna banku", "dane wrażliwe", "płatności cykliczne",
    "autoryzacja transakcji", "bezpieczeństwo danych", "alerty bankowe", "token", "kod PIN", "wyciąg bankowy",
    "potwierdzenie przelewu", "obsługa klienta", "reklamacje", "płatności NFC", "zakupy online", "cashback", "sesja Elixir",
    "kredyt studencki"
]

results = []

# Przetwarzanie listy tekstów
for text in texts:
    entities = model.predict_entities(text, labels)
    for entity in entities:
        results.append({
            "Text": text,
            "Entity": entity["text"],
            "Label": entity["label"]
        })

results_df = pd.DataFrame(results)
results_df


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\racha\Desktop\PROJECTS\PKO\analysis-advanced-chunking-techniques\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Text,Entity,Label
0,\n1. Użyte w Regulaminie określenia oznaczają:...,data waluty,waluty
1,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizje,prowizja
2,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizje,prowizja
3,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizje,prowizja
4,\n1. Za usługi świadczone przez PKO Bank Polsk...,opłaty,prowizja
5,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizji,prowizja


In [1]:
import pandas as pd
from gliner import GLiNER

# Załaduj model z Hugging Face
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

texts = [
"""
1. Użyte w Regulaminie określenia oznaczają:
1) dopuszczalne saldo debetowe – ujemne saldo rachunku oszczędnościowo-rozliczeniowego w wysokości nieprzekraczającej miesięcznego limitu ustalanego
przez PKO Bank Polski SA, z którego Posiadacz rachunku może korzystać na podstawie umowy rachunku,
2) data waluty – dzień, od którego lub do którego, naliczane są odsetki od środków pieniężnych, którymi został obciążony lub uznany rachunek,
3) dyspozycja – wszelkie oświadczenia woli lub wiedzy, w tym dotyczące operacji, składane przez Klienta,
4) dzień roboczy – dzień przypadający od poniedziałku do piątku, który nie jest dniem ustawowo wolnym od pracy,
5) kapitalizacja – dopisanie do salda rachunku naliczonych za dany okres odsetek od środków pieniężnych zgromadzonych na tym rachunku
"""
]

# Rozszerzona lista etykiet dla sektora bankowego
labels = [
    "kredyt hipoteczny", "pożyczka gotówkowa", "konto osobiste", "konto firmowe", "konto oszczędnościowe", "lokata terminowa",
    "ubezpieczenie", "kredyt firmowy", "kredyt konsolidacyjny", "karta kredytowa", "karta debetowa", "bankomat",
    "oddział bankowy", "opłaty", "prowizja", "transakcje", "limit kredytowy", "umowa kredytowa", "oprocentowanie",
    "kapitał", "odsetki", "raty", "czyszczenie historii kredytowej", "ubezpieczenie nieruchomości", "ubezpieczenie na życie",

]

results = []

# Przetwarzanie listy tekstów
for text in texts:
    entities = model.batch_predict_entities(text, labels, multi_label=True)
    for entity in entities:
        results.append({
            "Text": text,
            "Entity": entity["text"],
            "Label": entity["label"]
        })

# Przechowaj wyniki w DataFrame
results_df = pd.DataFrame(results)
print(results_df)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\racha\Desktop\PROJECTS\PKO\analysis-advanced-chunking-techniques\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

In [12]:
results_df

,Text,Entity,Label
0,\n1. Użyte w Regulaminie określenia oznaczają:...,data waluty,waluty
1,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizje,prowizja
2,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizje,prowizja
3,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizje,prowizja
4,\n1. Za usługi świadczone przez PKO Bank Polsk...,opłaty,prowizja
5,\n1. Za usługi świadczone przez PKO Bank Polsk...,opłaty,opłaty
6,\n1. Za usługi świadczone przez PKO Bank Polsk...,prowizji,prowizja
